# Configure settings

In [1]:
#@markdown ### General Config
MAX_SEQ_LENGTH =  1024#@param {type:"integer"}
PROCESSES = 2 #@param {type:"integer"}
NUM_TPU_CORES = 8
BUCKET_NAME = "theodore_jiang" #@param {type:"string"}
#@markdown ###### For if multiple models need to be evaluated at the same time: xxx is the placeholder for the individual model identifier (if only one is being evaluated xxx is the name of the model)
MODEL_NAME_FORMAT = "bert_model_xxx" #@param {type:"string"}
LOGGING_DIR_NAME_FORMAT = "bert_model_xxx_loss_spam" #@param {type:"string"}
PRETRAINING_DIR = "pretraining_data_1024" #@param {type:"string"}
EVAL_DIR = "eval_data_1024" #@param {type:"string"}
TESTING_DIR = "testing_data_1024" #@param {type:"string"}
RUN_NAME = "human_pretraining" #@param {type:"string"}
#import time
#RUN_NAME = RUN_NAME+"-"+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
VOC_FNAME = "vocab.txt" #@param {type:"string"}

#@markdown ### Input data pipeline config
MAX_PREDICTIONS = 20 #@param {type:"integer"}

#@markdown ### Training procedure config
EVAL_TEST_BATCH_SIZE = 64 #@param {type:"integer"}


#Mount Drive

In [2]:
from google.colab import drive
!fusermount -u /content/drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)
DRIVE_PATH = "/content/drive/My Drive"

fusermount: failed to unmount /content/drive: No such file or directory
Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


#Imports

In [3]:
 %tensorflow_version 1.x

import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import shutil
import time

print("current date/time:",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

if not os.path.exists("bert"):
  shutil.copytree("/content/drive/My Drive/bert-master","bert")
else:
  shutil.rmtree("bert")
  shutil.copytree("/content/drive/My Drive/bert-master","bert")
if "bert" in sys.path:
  sys.path.remove("bert")
sys.path.append("bert")

from bert import modeling, optimization, tokenization
from bert.modeling import BertModel,BertModelModified
from bert.run_pretraining import input_fn_builder, model_fn_builder

print("Authorize for GCS:")
auth.authenticate_user()
print("Authorize done")
  
# configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

log.handlers = []
#@markdown ###### Whether or not to write logs to a file
DO_FILE_LOGGING = False #@param {type:"boolean"}
if DO_FILE_LOGGING:
  #@markdown ###### If using file logging, what path to write logs to
  FILE_LOGGING_PATH = '/content/drive/My Drive/spam.log' #@param {type:"string"}
  fh = logging.FileHandler(FILE_LOGGING_PATH)
  fh.setLevel(logging.INFO)
  fh.setFormatter(formatter)
  log.addHandler(fh)

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
log.addHandler(ch)

if 'COLAB_TPU_ADDR' in os.environ:
  log.info("Using TPU runtime")
  USE_TPU = True
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(TPU_ADDRESS) as session:
    log.info('TPU address is ' + TPU_ADDRESS)
    # Upload credentials to TPU.
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    
else:
  log.warning('Not connected to TPU runtime')
  USE_TPU = False

TensorFlow 1.x selected.
current date/time: 2021-09-04 04:09:22

Authorize for GCS:
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



2021-09-04 04:13:40,845 - tensorflow - INFO - Using TPU runtime
2021-09-04 04:13:40,847 - tensorflow - INFO - TPU address is grpc://10.107.137.58:8470


Authorize done


In [4]:
#@markdown ###### To minimize interaction with GCS, the steps for each portion of the dataset are determined using the original data from the google drive
data_folder = "BERT pretraining/mutformer_pretraining_data" #@param {type: "string"}

data_path_eval = "/content/drive/My Drive/"+data_folder+"/sequences_"+str(MAX_SEQ_LENGTH)+"_eval.txt" 
data_path_test = "/content/drive/My Drive/"+data_folder+"/sequences_"+str(MAX_SEQ_LENGTH)+"_test.txt"

DATA_FPATH_eval = "dataset_eval.txt"
DATA_FPATH_test = "dataset_test.txt"

if os.path.exists(DATA_FPATH_eval):
  os.remove(DATA_FPATH_eval)
if os.path.exists(DATA_FPATH_test):
  os.remove(DATA_FPATH_test)

shutil.copy(data_path_eval,DATA_FPATH_eval)
shutil.copy(data_path_test,DATA_FPATH_test)

lines = open(DATA_FPATH_eval).read().split("\n")
EVAL_STEPS = len(lines)
lines = open(DATA_FPATH_test).read().split("\n")
TEST_STEPS = len(lines)
print("eval steps:",EVAL_STEPS,"test steps:",TEST_STEPS)

eval steps: 1000 test steps: 4554


## Data preparation

**TO ACCESS ANY BUCKET: go to this address: https://console.cloud.google.com/storage/browser/(BUCKET_NAME)**



#Parallel train-eval operation

In [5]:
def reload_ckpt(model_dir,logging_dir,current_ckpt,model,data_dir):
  BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, model_dir)

  VOCAB_FILE = os.path.join(BERT_GCS_DIR, VOC_FNAME)
  CONFIG_FILE = os.path.join(BERT_GCS_DIR, "config.json")

  INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR)
  print("init chkpt:",INIT_CHECKPOINT)
  print("current chkpt:",current_ckpt)
  if INIT_CHECKPOINT != current_ckpt:
    config = modeling.BertConfig.from_json_file(CONFIG_FILE)
    test_input_files = tf.gfile.Glob(os.path.join(data_dir,'*tfrecord'))
    log.info("Using {} data shards for testing".format(len(test_input_files)))
    model_fn = model_fn_builder(
          bert_config=config,
          logging_dir=logging_dir,
          init_checkpoint=INIT_CHECKPOINT,
          init_learning_rate=1,
          decay_per_step=1,
          num_warmup_steps=10,
          use_tpu=USE_TPU,
          use_one_hot_embeddings=True,
          bert=model)

    
    if USE_TPU:
      tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

      run_config = tf.contrib.tpu.RunConfig(
          cluster=tpu_cluster_resolver,
          model_dir=BERT_GCS_DIR,
          save_checkpoints_steps=1,
          tpu_config=tf.contrib.tpu.TPUConfig(
              iterations_per_loop=1,
              num_shards=NUM_TPU_CORES,
              per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

      estimator = tf.contrib.tpu.TPUEstimator(
          use_tpu=USE_TPU,
          model_fn=model_fn,
          config=run_config,
          train_batch_size=1,
          eval_batch_size=EVAL_TEST_BATCH_SIZE)
    else:
      run_config = tf.estimator.RunConfig(
          model_dir=BERT_GCS_DIR,
          save_checkpoints_steps=1)


      estimator = tf.contrib.tpu.TPUEstimator(
          use_tpu=USE_TPU,
          model_fn=model_fn,
          config=run_config,
          train_batch_size=1,
          eval_batch_size=EVAL_TEST_BATCH_SIZE)

    test_input_fn = input_fn_builder(
            input_files=test_input_files,
            max_seq_length=MAX_SEQ_LENGTH,
            max_predictions_per_seq=MAX_PREDICTIONS,
            is_training=False)
    return INIT_CHECKPOINT,estimator,test_input_fn,True
  else:
    return None,None,None,False

#Evaluation

In [ ]:
import time
BUCKET_PATH = "gs://{}".format(BUCKET_NAME)
#@markdown ###### whether to evaluate on the test set or the dev set ("test" or "dev") (test set will only run once, dev set will run continuously)
dataset = "dev" #@param{type:"string"}

if dataset=="test":
  DATA_DIR = TESTING_DIR
elif dataset=="dev":
  DATA_DIR = EVAL_DIR
else:
  raise Exception("only datasets supported are dev and test")

models_to_evaluate = ["modified","orig","large","modified_medium","modified_large"] ##list of models to evaluate

name2model = {      ##dictionary corresponding model architecture to each model name
    "modified":BertModelModified,
    "modified_medium":BertModelModified,
    "modified_large":BertModelModified,
    "orig":BertModel,
    "large":BertModel
}

def write_metrics(metrics,dir): ##evaluation metrics will be written into google drive to minimize interations with GCS
  summary_map = {"eval_f1":metrics["multiclass dice/f1"],     ##maps a name of a metric from the source code to a name to display
               "eval_accuracy":metrics["masked_lm_accuracy"], ##(only change if name of metrics were changed in source code)
               "eval_precision":metrics["precision"],
               "eval_recall":metrics["recall"],
               "eval_loss":metrics["loss"]}
  gs = metrics["global_step"]
  print("global step",gs)

  tf.compat.v1.disable_eager_execution()
  tf.reset_default_graph()  
  for key,value in summary_map.items():
    print(key,value)
    x_scalar = tf.constant(value)
    first_summary = tf.summary.scalar(name=key, tensor=x_scalar)

    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        writer = tf.summary.FileWriter(dir)
        sess.run(init)
        summary = sess.run(first_summary)
        writer.add_summary(summary, gs)
        writer.flush()
        print('Done with writing the scalar summary')
    time.sleep(1)
  if not os.path.exists(DRIVE_PATH+"/"+dir):
    os.makedirs(DRIVE_PATH+"/"+dir)
  cmd = "cp -r \""+dir+"/.\" \""+DRIVE_PATH+"/"+dir+"\""
  !{cmd}
current_ckpts = ["N/A" for i in range(len(models_to_evaluate))]

while True:
  for n,model in enumerate(models_to_evaluate):
    MODEL_DIR = MODEL_NAME_FORMAT.replace("xxx",model)
    GCS_LOGGING_DIR = "{}/{}".format(LOGGING_DIR_NAME_FORMAT.replace("xxx",model),RUN_NAME)
    current_ckpt = current_ckpts[n]
    current_ckpt,estimator,test_input_fn,new = reload_ckpt(MODEL_DIR,GCS_LOGGING_DIR,current_ckpt,name2model[model],"{}/{}".format(BUCKET_PATH, DATA_DIR))
    current_ckpts[n] = current_ckpt
    if new:
      print("\n\nEVALUATING "+model+" MODEL\n\n")
      log.info("Using checkpoint: {}".format(current_ckpt))
      metrics = estimator.evaluate(input_fn=test_input_fn, steps=int(TEST_STEPS/EVAL_TEST_BATCH_SIZE))
      write_metrics(metrics,GCS_LOGGING_DIR)

  print("finished 1 eval loop")
  if dataset=="test":
    break
  time.sleep(600)


2021-09-04 04:13:44,724 - tensorflow - WARNING - From /content/bert/modeling.py:96: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



init chkpt: gs://theodore_jiang/bert_model_modified/model.ckpt-1504378
current chkpt: N/A


2021-09-04 04:13:44,979 - tensorflow - INFO - Using 1 data shards for testing
2021-09-04 04:13:45,986 - tensorflow - WARNING - Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f248bc2ec20>) includes params argument, but params are not passed to Estimator.
2021-09-04 04:13:45,988 - tensorflow - INFO - Using config: {'_model_dir': 'gs://theodore_jiang/bert_model_modified', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.107.137.58:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_se



EVALUATING modified MODEL




2021-09-04 04:13:46,423 - tensorflow - WARNING - From /tensorflow-1.15.2/python3.7/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2021-09-04 04:13:46,428 - tensorflow - INFO - Calling model_fn.
2021-09-04 04:13:46,434 - tensorflow - WARNING - From /content/bert/run_pretraining.py:314: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.

2021-09-04 04:13:46,462 - tensorflow - WARNING - From /content/bert/run_pretraining.py:360: map_and_batch (from tensorflow.contrib.data.python.ops.batching) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
2021-09-04 04:13:46,463 - tensorflow - WARNING - From /tensorflow-1.15.2/python3.7/tensorf

...

2021-09-04 04:41:27,658 - tensorflow - INFO - Calling model_fn.
2021-09-04 04:41:27,690 - tensorflow - WARNING - Entity <function input_fn_builder.<locals>.input_fn.<locals>.<lambda> at 0x7f246fd74cb0> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: module 'gast' has no attribute 'Str'
2021-09-04 04:41:27,794 - tensorflow - INFO - *** Features ***
2021-09-04 04:41:27,795 - tensorflow - INFO -   name = input_ids, shape = (8, 1024)
2021-09-04 04:41:27,799 - tensorflow - INFO -   name = input_mask, shape = (8, 1024)
2021-09-04 04:41:27,808 - tensorflow - INFO -   name = masked_lm_ids, shape = (8, 20)
2021-09-04 04:41:27,810 - tensorflow - INFO -   name = masked_lm_positions, shape = (8, 20)
2021-09-04 04:41:27,811 - tensorflow - INFO -   name = masked_lm_weights, shape = (8, 20)
2021-09-04 04:41:27,813 - tensorflow - INF

embedding shape: (8, 1024, 768)
embedding shape: (8, 1024, 768)


2021-09-04 04:41:30,738 - tensorflow - INFO - **** Trainable Variables ****
2021-09-04 04:41:30,755 - tensorflow - INFO -   name = bert/embeddings/word_embeddings:0, shape = (27, 768), *INIT_FROM_CKPT*
2021-09-04 04:41:30,757 - tensorflow - INFO -   name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
2021-09-04 04:41:30,759 - tensorflow - INFO -   name = bert/embeddings/position_embeddings:0, shape = (1024, 768), *INIT_FROM_CKPT*
2021-09-04 04:41:30,761 - tensorflow - INFO -   name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
2021-09-04 04:41:30,763 - tensorflow - INFO -   name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
2021-09-04 04:41:30,764 - tensorflow - INFO -   name = bert/embeddings/conv1d/kernel:0, shape = (3, 768, 768), *INIT_FROM_CKPT*
2021-09-04 04:41:30,765 - tensorflow - INFO -   name = bert/embeddings/conv1d/bias:0, shape = (768,), *INIT_FROM_CKPT*
2021-09-04 04:41:30,766 - tensorflow - INFO

(1280, 27) (1280, 27) (1280, 1)


2021-09-04 04:41:35,692 - tensorflow - INFO - Done calling model_fn.
2021-09-04 04:41:35,713 - tensorflow - INFO - Starting evaluation at 2021-09-04T04:41:35Z
2021-09-04 04:41:35,717 - tensorflow - INFO - TPU job name worker
2021-09-04 04:41:36,134 - tensorflow - INFO - Graph was finalized.
2021-09-04 04:41:36,369 - tensorflow - INFO - Restoring parameters from gs://theodore_jiang/bert_model_modified/model.ckpt-1504378
2021-09-04 04:41:37,690 - tensorflow - INFO - Running local_init_op.
2021-09-04 04:41:37,865 - tensorflow - INFO - Done running local_init_op.
2021-09-04 04:41:38,262 - tensorflow - INFO - Init TPU system


In [ ]:
ndomspamfo#randomspamforpreventirpreventi